In [20]:
# Common imports and settings
import os, sys, re
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr
import dask
from dask.distributed import Client
from dask_gateway import Gateway
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import numpy as np
# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                                  # https://github.com/opendatacube/odc-geo
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool                  # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from dea_tools.plotting import display_map, rgb    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

import boto3

# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Holoviews
# https://holoviz.org/tutorial/Composing_Plots.html
# https://holoviews.org/user_guide/Composing_Elements.html
import hvplot.pandas
import hvplot.xarray
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# hv.extension('bokeh', logo=False)
print("Libraries loaded successfully.")

Libraries loaded successfully.


In [21]:
# EASI defaults
# These are convenience functions so that the notebooks in this repository work in all EASI deployments

# The `git.Repo()` part returns the local directory that easi-notebooks has been cloned into
# If using the `easi-tools` functions from another path, replace `repo` with your local path to `easi-notebooks` directory
try:
    import git
    repo = git.Repo('.', search_parent_directories=True).working_tree_dir    # Path to this cloned local directory
except (ImportError, git.InvalidGitRepositoryError):
    repo = Path.home() / 'easi-notebooks'    # Reasonable default
    if not repo.is_dir():
        raise RuntimeError('To use `easi-tools` please provide the local path to `https://github.com/csiro-easi/easi-notebooks`')
if repo not in sys.path:
    sys.path.append(str(repo))    # Add the local path to `easi-notebooks` to python

from easi_tools import EasiDefaults
from easi_tools import initialize_dask, xarray_object_size, mostcommon_crs, heading
#from easi_tools.load_s2l2a import load_s2l2a_with_offset
print("EASI librariies loaded successfully.")

EASI librariies loaded successfully.


In [22]:
import planetary_computer
from pystac_client import Client as PClient
import odc.stac
import boto3
import warnings
warnings.filterwarnings('ignore')
from odc.geo.geobox import GeoBox
from odc.geo.geom import box as odc_box
import time as _time
from pystac_client.exceptions import APIError

import zarr

print("Data Loader Libraries Imported Successfully")

Data Loader Libraries Imported Successfully


In [23]:
import torch
from terratorch.registry import BACKBONE_REGISTRY

#from dask_pytorch_ddp import dispatch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.multiprocessing as mp
import torch.optim as optim
from datetime import datetime#, timedelta
import json
import torch.distributed as dist
import terratorch
from contextlib import nullcontext
from torch.cuda.amp import autocast, GradScaler
from s3torchconnector import S3Checkpoint
print("DL libraries imported successfully")

DL libraries imported successfully


In [24]:
import icechunk as ic
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
from nvidia.dali.plugin.pytorch import DALIGenericIterator


print("GPU Libraries imported successfully")

GPU Libraries imported successfully


In [25]:
gateway = Gateway()
options = gateway.cluster_options()
# To get GPUs you must specify
options.cuda_worker = True
clusters = gateway.list_clusters()
if not clusters:
    print('Creating new cluster...')
    cluster = gateway.new_cluster(cluster_options=options)
else:
    print(f'Connecting to existing cluster: {clusters[0].name}')
    cluster = gateway.connect(clusters[0].name)

NUM_WORKERS = 4
cluster.scale(NUM_WORKERS)
client = cluster.get_client()

print("Cluster Dashboard:", client.dashboard_link)

Connecting to existing cluster: easihub.494fb1b8cc9f4b4a926d0149ba92e2dc
Cluster Dashboard: https://hub.csiro.easi-eo.solutions/services/dask-gateway/clusters/easihub.494fb1b8cc9f4b4a926d0149ba92e2dc/status


In [26]:
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

In [27]:
# === S3 SETUP ===
easi = EasiDefaults()
bucket = easi.scratch
userid = boto3.client('sts').get_caller_identity()['UserId']
project_name = "training_test_project"
base_prefix = f"{userid}/{project_name}"
s3_base = f"s3://{bucket}/{base_prefix}"

repo_prefix="AROAWO7MSC2T2SINGSYOQ:csiro-csiro-aad_gre555@csiro.au/training_test_project/training_dataset_v4-icechunk"
snapshot_id="7HCZNF9XRHCY6M6VCJN0"


Successfully found configuration for deployment "csiro"


In [90]:

print("Installing icechunk (and zarr>=3) on all workers...")
client.run(lambda: exec(
    "import sys, subprocess; "
    "subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'zarr>=3.0.8', 'icechunk'])"
))

print("Installing icechunk (and zarr>=3) on the Scheduler...")
client.run_on_scheduler(lambda: exec(
    "import sys, subprocess; "
    "subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'zarr>=3.0.8', 'icechunk'])"
))

print("Installation initiated on both Scheduler and Workers. Now retry the Icechunk build.")



print("Installing Dali on all workers...")
client.run(lambda: exec(
    "import sys, subprocess; "
    "subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nvidia-dali-cuda120'])"
))

print("Installing Dali on the Scheduler...")
client.run_on_scheduler(lambda: exec(
    "import sys, subprocess; "
    "subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nvidia-dali-cuda120'])"
))

print("Installation initiated on both Scheduler and Workers. Now retry the Dali build.")


Installing icechunk (and zarr>=3) on all workers...
Installing icechunk (and zarr>=3) on the Scheduler...
Installation initiated on both Scheduler and Workers. Now retry the Icechunk build.
Installing Dali on all workers...
Installing Dali on the Scheduler...
Installation initiated on both Scheduler and Workers. Now retry the Dali build.


In [29]:
class LabelMapper:
    def map(self, raw_labels: np.ndarray) -> np.ndarray:
        raise NotImplementedError
class WorldCoverLabelMapper(LabelMapper):
    def __init__(self, num_classes=11, ignore_index=255):
        self.num_classes = num_classes
        self.ignore_index = ignore_index
        self.code_map = {
            10: 0, 20: 1, 30: 2, 40: 3, 50: 4,
            60: 5, 70: 6, 80: 7, 90: 8, 95: 9, 100: 10,
        }

    def map(self, raw_labels: np.ndarray) -> np.ndarray:
        y = raw_labels.copy()
        y_mapped = np.full_like(y, fill_value=self.ignore_index, dtype=np.int64)
        for wc_code, cls_idx in self.code_map.items():
            y_mapped[y == wc_code] = cls_idx
        y_mapped[np.isnan(y)] = self.ignore_index
        return y_mapped.astype(np.int64)

# NUMPY Changes
Needed to change numpy version to <2 to get this to work

In [72]:
from torch.utils.data import get_worker_info
import cupy as cp
import torch.utils.dlpack as dlpack
import torch
import numpy as np

import zarr 


class ZarrGPUWorldCover(torch.utils.data.Dataset):
    def __init__(self, bucket, repo_prefix, snapshot_id=None,
                 branch="main", label_mapper=None, region="ap-southeast-2"):
        self.bucket = bucket
        self.repo_prefix = repo_prefix
        self.snapshot_id = snapshot_id
        self.branch = branch
        self.region = region
        self.label_mapper = label_mapper or WorldCoverLabelMapper()
        self._feats = None
        self._labels = None
        self._n = None

    def _lazy_init(self):
        if self._feats is not None:
            return
        storage = ic.s3_storage(bucket=self.bucket, prefix=self.repo_prefix,
                                region=self.region, from_env=True)
        repo = ic.Repository.open(storage)
        if self.snapshot_id is not None:
            sess = repo.readonly_session(snapshot_id=self.snapshot_id)
        else:
            sess = repo.readonly_session(branch=self.branch)

        # Enable GPU-backed buffers at the Zarr level [web:25]
        with zarr.config.enable_gpu():
            root = zarr.open_group(store=sess.store, mode="r")
            self._feats = root["features"]   # Zarr arrays
            self._labels = root["labels"]
        self._n = self._feats.shape[0]

    def __len__(self):
        self._lazy_init()
        return self._n

    def __getitem__(self, idx):
        self._lazy_init()

        # 1. Features: Zarr → CuPy (GPU) → DLPack → torch.cuda.Tensor
        x_cu = self._feats[idx]              # cupy.ndarray if enable_gpu worked
        if not isinstance(x_cu, cp.ndarray):
            # Fallback: NumPy → torch
            x = torch.from_numpy(np.asarray(x_cu)).float()
        else:
            x = dlpack.from_dlpack(x_cu.toDlpack()).float()

        # 2. Labels: small, keep them CPU→GPU
        y_raw = self._labels[idx]           # NumPy
        y_np = self.label_mapper.map(y_raw)
        y = torch.from_numpy(y_np).long()

        return {"inputs": x, "labels": y, "meta": {"index": idx}}




class IcechunkBatchIterator:
    def __init__(self, bucket, repo_prefix, snapshot_id,
                 batch_size, device, indices=None, shuffle=True,
                 region="ap-southeast-2"):
        storage = ic.s3_storage(bucket=bucket,
                                prefix=repo_prefix,
                                region=region,
                                from_env=True)
        repo = ic.Repository.open(storage)
        session = repo.readonly_session(snapshot_id=snapshot_id)

        # Direct Zarr v3 + GPU buffers, no Xarray [web:25]
        with zarr.config.enable_gpu():
            root = zarr.open_group(store=session.store, mode="r")
            self.feats = root["features"]   # (sample, band, time, y, x)
            self.labels = root["labels"]    # (sample, y, x)

        n = self.feats.shape[0]
        if indices is None:
            indices = np.arange(n)
        self.indices = np.array(indices)

        self.batch_size = batch_size
        self.device = device
        self.shuffle = shuffle
        self.n = len(self.indices)
        self.i = 0

    def __iter__(self):
        self.i = 0
        if self.shuffle:
            np.random.shuffle(self.indices)
        return self
        
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
    
        start = self.i
        end   = min(self.i + self.batch_size, self.n)
        self.i = end
        idxs = np.arange(start, end)
    
        x_list = []
        for idx in idxs:
            x_i = self.feats[int(idx)]        # may be CuPy *or* NumPy
            # print("DEBUG type(x_i):", type(x_i))
            x_list.append(x_i)
    
        if isinstance(x_list[0], cp.ndarray):
            # All elements should be same type; stack on GPU and use DLPack
            x_cu = cp.stack(x_list, axis=0)
            x = dlpack.from_dlpack(x_cu.toDlpack()).float()
        else:
            # CPU fallback: stack NumPy, then move to GPU
            x_np = np.stack([np.asarray(a) for a in x_list], axis=0)
            x = torch.from_numpy(x_np).float().to(self.device, non_blocking=True)
    
        y_raw = self.labels[start:end]            # NumPy
        y_np = np.stack([WorldCoverLabelMapper().map(lbl) for lbl in y_raw])
        y = torch.from_numpy(y_np).long().to(self.device, non_blocking=True)
    
        return {"inputs": x, "labels": y, "meta": {"indices": idxs}}


In [74]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
from nvidia.dali.plugin.pytorch import DALIGenericIterator

class IcechunkExternalInput:
    """
    Simple wrapper that makes your iterator usable by DALI ExternalSource.
    """
    def __init__(self, base_iterator):
        self.base_iterator = base_iterator

    def __iter__(self):
        # Re-create a fresh underlying iterator each epoch
        self.it = iter(self.base_iterator)
        return self

    def __next__(self):
        batch = next(self.it)  # may raise StopIteration
        x = batch["inputs"]    # torch.Tensor (CPU or CUDA)
        y = batch["labels"]    # torch.Tensor

        # DALI expects arrays; give it CPU NumPy
        x_np = x.detach().cpu().numpy()
        y_np = y.detach().cpu().numpy()
        return x_np, y_np


In [75]:
@pipeline_def
def dali_pipeline(eii):
    # External source provides (images, labels) per batch
    images, labels = fn.external_source(
        source=eii,
        num_outputs=2,
        device="cpu",  # input is CPU NumPy
    )
    # Move images to GPU inside DALI; labels can stay CPU or go GPU as needed
    images = images.gpu()  # now on device_id GPU [web:31]

    # You could add more GPU ops here: resize, normalize, etc.
    # images = fn.crop_mirror_normalize(images, ...)
    return images, labels
    
def make_dali_iterator(bucket, repo_prefix, snapshot_id, batch_size, device_id=0):
    base_it = IcechunkBatchIterator(
        bucket=bucket,
        repo_prefix=repo_prefix,
        snapshot_id=snapshot_id,
        batch_size=batch_size,
        device=torch.device("cpu"),  # we hand CPU batches to DALI here
        shuffle=True,
    )
    eii = IcechunkExternalInput(base_it)

    pipe = dali_pipeline(
        eii=eii,
        batch_size=batch_size,
        num_threads=4,
        device_id=device_id,
    )
    pipe.build()

    dali_it = DALIGenericIterator(
        [pipe],
        output_map=["inputs", "labels"],
        auto_reset=True,
    )
    return dali_it

In [87]:
def remote_test_dataset(bucket, repo_prefix, snapshot_id=None, branch="main"):
    import time, torch
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = ZarrGPUWorldCover(bucket, repo_prefix, snapshot_id=snapshot_id, branch=branch)
    dl = DataLoader(ds, batch_size=8, shuffle=True, num_workers=0)

    t0 = time.perf_counter()
    n = 0
    for i, batch in enumerate(dl):
        x = batch["inputs"].to(DEVICE, non_blocking=True)
        y = batch["labels"].to(DEVICE, non_blocking=True)
        _ = x.sum()
        n += x.shape[0]
        if i >= 10:
            break
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    return {"samples": n, "seconds": t1 - t0}

# run on one GPU worker
f = client.submit(remote_test_dataset, bucket, repo_prefix, snapshot_id)
print(f.result())

KeyboardInterrupt: 

In [86]:
f = client.submit(remote_test_dataset, bucket, repo_prefix, snapshot_id, pure=False)
print("status:", f.status)

try:
    res = f.result()   # this will raise if the worker crashed in that function
    print("result:", res)
except Exception as e:
    print("REMOTE ERROR:", e)
    import traceback
    traceback.print_tb(f.traceback())



status: pending
result: {'samples': 88, 'seconds': 17.965287756999714}


In [88]:
def remote_test_iterator(bucket, repo_prefix, snapshot_id=None):
    import time, torch
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    it = IcechunkBatchIterator(bucket, repo_prefix, snapshot_id,
                               batch_size=8, device=DEVICE, shuffle=True)
    t0 = time.perf_counter()
    n = 0
    for i, batch in enumerate(it):
        x = batch["inputs"]
        _ = x.sum()
        n += x.shape[0]
        if i >= 10:
            break
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    return {"samples": n, "seconds": t1 - t0}

f_it = client.submit(remote_test_iterator, bucket, repo_prefix, snapshot_id)
print(f_it.result())



{'samples': 88, 'seconds': 13.202695846000097}


In [93]:
def remote_test_dali(bucket, repo_prefix, snapshot_id=None):
    import time, torch
    import torch.nn as nn
    import torch.optim as optim

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class SmallModel(nn.Module):
        def __init__(self, in_channels=6):
            super().__init__()
            self.conv = nn.Conv2d(in_channels=in_channels * 3, out_channels=16, kernel_size=3, padding=1)
            self.relu = nn.ReLU()
            self.head = nn.Conv2d(16, 11, kernel_size=1)

        def forward(self, x):
            B, Bn, T, H, W = x.shape
            x = x.view(B, Bn * T, H, W)
            x = self.relu(self.conv(x))
            return self.head(x)

    batch_size = 8
    dali_it = make_dali_iterator(bucket, repo_prefix, snapshot_id,
                                 batch_size=batch_size, device_id=0)

    model = SmallModel(in_channels=6).to(DEVICE)
    opt = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    t0 = time.perf_counter()
    n = 0
    for i, data in enumerate(dali_it):
        x = data[0]["inputs"].to(DEVICE, non_blocking=True)
        y = data[0]["labels"].to(DEVICE, non_blocking=True)

        opt.zero_grad()
        logits = model(x)
        loss = criterion(logits.view(logits.size(0), logits.size(1), -1),
                         y.view(y.size(0), -1))
        loss.backward()
        opt.step()

        n += x.shape[0]
        if i >= 20:
            break
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    return {"samples": n, "seconds": t1 - t0}
client.restart()
f_dali = client.submit(remote_test_dali, bucket, repo_prefix, snapshot_id)
print(f_dali.result())


{'samples': 168, 'seconds': 24.653500735000307}


In [68]:
import icechunk as ic
storage = ic.s3_storage(bucket=bucket, prefix=repo_prefix, region="ap-southeast-2", from_env=True)
repo = ic.Repository.open(storage)
session = repo.readonly_session(branch="main")  # or snapshot_id=...

ds = xr.open_zarr(
    session.store,
    zarr_format=3,
    consolidated=False,
)

print(ds)                 # should print without error
print(ds.data_vars)       # confirm 'features' and 'labels' are present
print(ds.sizes["sample"])


<xarray.Dataset> Size: 1GB
Dimensions:      (sample: 627, band: 6, time: 3, y: 224, x: 224)
Coordinates:
    spatial_ref  int32 4B ...
  * sample       (sample) int64 5kB 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23 24
  * x            (x) int64 2kB 0 1 2 3 4 5 6 7 ... 217 218 219 220 221 222 223
  * y            (y) int64 2kB 0 1 2 3 4 5 6 7 ... 217 218 219 220 221 222 223
Dimensions without coordinates: band, time
Data variables:
    features     (sample, band, time, y, x) uint16 1GB dask.array<chunksize=(8, 6, 3, 224, 224), meta=np.ndarray>
    labels       (sample, y, x) uint8 31MB dask.array<chunksize=(8, 224, 224), meta=np.ndarray>
Attributes:
    bands:         ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    n_time_steps:  3
    regions_done:  ['nsw_coast_n', 'wa_arid_n', 'nsw_coast_e', 'wa_arid_s', '...
Data variables:
    features  (sample, band, time, y, x) uint16 1GB dask.array<chunksize=(8, 6, 3, 224, 224), meta=np.ndarray>
    labels    (sample, y, x) uint8 31MB dask.arr

In [69]:
import icechunk as ic
import zarr
import cupy as cp

storage = ic.s3_storage(bucket=bucket, prefix=repo_prefix,
                        region="ap-southeast-2", from_env=True)
repo = ic.Repository.open(storage)
session = repo.readonly_session(branch="main")

with zarr.config.enable_gpu():
    root = zarr.open_group(store=session.store, mode="r")
    feats_za = root["features"]  # (sample, band, time, y, x)
    labels_za = root["labels"]

    arr0 = feats_za[0]           # one sample
    print("DEBUG root['features'][0] type:", type(arr0))


DEBUG root['features'][0] type: <class 'cupy.ndarray'>


In [45]:
import icechunk as ic
import time as time
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
def test_dataset():
    ds = ZarrGPUWorldCover(bucket, repo_prefix, snapshot_id=None, branch="main")
    dl = DataLoader(ds, batch_size=8, shuffle=True, num_workers=0)

    print("Testing ZarrGPUWorldCover...")
    t0 = time.perf_counter()
    n = 0
    for i, batch in enumerate(dl):
        x = batch["inputs"].to(DEVICE, non_blocking=True)
        y = batch["labels"].to(DEVICE, non_blocking=True)
        _ = x.sum()
        n += x.shape[0]
        if i >= 10:
            break
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    print(f"saw {n} samples in {t1 - t0:.2f}s → {n/(t1 - t0):.1f} samples/s")



#dask.config.set({"array.backend": "cupy"})
#zarr.config.enable_gpu()
client.restart()
test_dataset()

Using device: cuda
Testing ZarrGPUWorldCover...
saw 88 samples in 14.25s → 6.2 samples/s


In [73]:
def test_iterator():
    it = IcechunkBatchIterator(bucket, repo_prefix, snapshot_id,
                               batch_size=8, device=DEVICE)
    print("Testing IcechunkBatchIterator...")
    t0 = time.perf_counter()
    n = 0
    for i, batch in enumerate(it):
        x = batch["inputs"]
        y = batch["labels"]
        a = x.sum()
        n += x.shape[0]
        if i >= 10:
            break
    torch.cuda.synchronize()
    t1 = time.perf_counter()
    print(f"saw {n} samples in {t1 - t0:.2f}s  → {n/(t1 - t0):.1f} samples/s")
client.restart()
test_iterator()

Testing IcechunkBatchIterator...
saw 88 samples in 12.21s  → 7.2 samples/s


In [23]:
from numba import cuda
from dask.distributed import Client


def cuda_runtime_version():
    # Returns (major, minor) from the CUDA runtime
    return cuda.runtime.get_version()

versions_by_worker = client.run(cuda_runtime_version)
print(versions_by_worker)



{'tls://10.0.79.52:46183': (12, 8), 'tls://10.0.79.53:45653': (12, 8)}


BROKEN env:
numpy    : 2.2.6
xarray   : 2025.9.0
zarr     : 3.1.2
numcodecs: 0.16.2


In [81]:
client.restart()

SyntaxError: invalid syntax (2190545615.py, line 10)

In [ ]:
client.close()
cluster.shutdown()